In [1]:
import codecs
import re

In [2]:
def load_file(file_path):
    f = codecs.open(file_path, 'r', 'utf-8')
    text = f.readlines()
    text = ''.join(text)
    return text

In [3]:
def export_to_srt(file_path):
    with codecs.open(file_path, 'w+', 'utf-8') as f:
        for i in range(len(users)):
            f.write(f"{str(i+1)}\n")
            f.write(f"{new_timestamps[i]}\n")
            f.write(f"{users[i]}: {comments[i]}\n\n")
        f.close()


In [4]:
text = load_file('meeting_saved_chat.txt')

In [5]:
print(text)

08:02:35 From  Host  to  Everyone:
	Yang mau request lagu boleh nih
08:03:12 From  Participant1  to  Everyone:
	opick-tombo ati *****
08:03:16 From  Participant3  to  Everyone:
	cocok
08:03:37 From  Host  to  Everyone:
	Okeeh
08:09:31 From  Participant1  to  Everyone:
	pagi  *****
08:09:40 From  Participant2  to  Everyone:
	pagi *****
08:10:36 From  Participant4  to  Everyone:
	pagi *****
08:10:55 From  Participant5  to  Everyone:
	pernah
08:11:31 From  Participant5  to  Everyone:
	semacam hosting gitu ***** wkwk
08:12:38 From  Participant6  to  Everyone:
	heroku itu adalah web hosting berbasis cloud *****
08:16:21 From  Host  to  Everyone:
	remove bg
08:16:26 From  Participant7  to  Everyone:
	pernah *****
08:16:27 From  Participant2  to  Everyone:
	pernah *****
08:16:28 From  Participant8  to  Everyone:
	tau *****
08:16:28 From  Participant9  to  Everyone:
	iya menghapus backgroud
08:16:28 From  Participant9  to  Everyone:
	tau *****
08:16:29 From  Participant10  to  Everyone:
	perna

In [6]:
timestamps = re.findall("(\d{2}:\d{2}:\d{2}) From", text)

In [7]:
messages = re.split("\d{2}:\d{2}:\d{2} From [ \S]+to [ \S]+:", text)
messages.pop(0) # remove the first empty element
messages = [m.strip() for m in messages]

In [8]:
users = re.findall("\d{2}:\d{2}:\d{2} From ([ \S]+)to [ \S]+:", text)
users = [user.strip() for user in users]

## Problem Understanding
*TODO:* 
- convert timestamps zoom ke timestamp video
- tampilkan message dengan durasi yang sesuai


In [9]:
# 5 kata per detik.
# 200ms per kata
# https://stackoverflow.com/questions/43305577/python-calculate-the-difference-between-two-datetime-time-objects
from datetime import datetime, date, time, timedelta


In [10]:
# TODO: add this to a class attribute

# reset the hours to 0 because the chat timestamp start with the clock time,
# not the video's timestamp
start_hour = int(timestamps[0].split(":")[0])

# input: datetime obj
# return: resetted datetime obj
def reset_hour(datetime_obj):
    datetime_obj -= timedelta(hours=start_hour)
    return datetime_obj

In [11]:
# input: text
# return: datetime object
def txt_to_time(text):
    h, m, s = (int(part) for part in text.split(":"))
    t = time(h,m,s)
    print(t)
    return datetime.combine(date.today(), t)

In [12]:
# input:t1,t2 => datetime object
# return: diff time in ms
def diff_time(t1, t2):
    diff = t2 - t1
    return diff.seconds

In [13]:
timestamps[0], timestamps[30]

('08:02:35', '08:16:38')

In [14]:
t0 = txt_to_time(timestamps[0])
t0

08:02:35


datetime.datetime(2022, 6, 16, 8, 2, 35)

In [15]:
z0 = reset_hour(t0)

In [16]:
z0

datetime.datetime(2022, 6, 16, 0, 2, 35)

In [17]:
tn = txt_to_time(timestamps[39])
tn
zn = reset_hour(tn)

08:42:20


In [18]:
zn

datetime.datetime(2022, 6, 16, 0, 42, 20)

In [19]:
diff_time(t0, tn)

2385